In [70]:
import numpy as np
import pandas as pd
import math
import csv

In [71]:
data = pd.read_csv('titanic.csv')

In [72]:
data

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
882,0,2,Rev. Juozas Montvila,male,27.0,0,0,13.0000
883,1,1,Miss. Margaret Edith Graham,female,19.0,0,0,30.0000
884,0,3,Miss. Catherine Helen Johnston,female,7.0,1,2,23.4500
885,1,1,Mr. Karl Howell Behr,male,26.0,0,0,30.0000


In [73]:
survivedlist = ['Yes' if Survived == 1 else 'No' for Survived in data.iloc[:]['Survived'].values]

In [74]:
pclasslist = []
for i in data.iloc[:]['Pclass'].values:
    if i == 1:
        pclasslist.append('First')
    elif i == 2:
        pclasslist.append('Second')
    else:
        pclasslist.append('Third')

In [75]:
genderList = ['Male' if gender == 'male' else 'Female' for gender in data.iloc[:]['Sex'].values]

In [76]:
agelist = []
for age in data.iloc[:]['Age']:
    if age < 18:
        agelist.append('Child')
    elif age > 18 and age <=50:
        agelist.append('Adult')
    else:
        agelist.append('Senior_citizen')

In [77]:
data.rename(columns = {'Siblings/Spouses Aboard' : 'SiblingsSpousesAboard' }, inplace = True)
sslist = ['Ssayes' if SiblingsSpousesAboard == 1 else 'Ssano' for SiblingsSpousesAboard in data.iloc[:]['SiblingsSpousesAboard'].values]

In [78]:
data.rename(columns = {'Parents/Children Aboard' : 'ParentsChildrenAboard' }, inplace = True)
plist = ['Pcano' if ParentsChildrenAboard == 0 else 'Pcayes' for ParentsChildrenAboard in data.iloc[:]['ParentsChildrenAboard'].values]

In [83]:
farelist = []
for fare in data.iloc[:]['Fare'].values:
    if fare <= 50:
        farelist.append('Low')
    elif fare > 50 and fare <= 200:
        farelist.append('Moderate')
    else:
        farelist.append('High')

In [84]:
processed_data = pd.DataFrame()

In [85]:
processed_data['pclass'] = pclasslist
processed_data['gender'] = genderList
processed_data['age'] = agelist
processed_data['ssa'] = sslist
processed_data['pca'] = plist
processed_data['fare'] = farelist
processed_data['survived'] = survivedlist

In [86]:
processed_data

,pclass,gender,age,ssa,pca,fare,survived
0,Third,Male,Adult,Ssayes,Pcano,Low,No
1,First,Female,Adult,Ssayes,Pcano,Moderate,Yes
2,Third,Female,Adult,Ssano,Pcano,Low,Yes
3,First,Female,Adult,Ssayes,Pcano,Moderate,Yes
4,Third,Male,Adult,Ssano,Pcano,Low,No
...,...,...,...,...,...,...,...
882,Second,Male,Adult,Ssano,Pcano,Low,No
883,First,Female,Adult,Ssano,Pcano,Low,Yes
884,Third,Female,Child,Ssayes,Pcayes,Low,No
885,First,Male,Adult,Ssano,Pcano,Low,Yes


In [88]:
processed_data.to_csv('new_titanic.csv', index=False)

In [38]:
def load_csv(filename):
    lines = csv.reader(open(filename,'r'))
    dataset = list(lines)
    headers = dataset.pop(0)
    return dataset, headers

In [26]:
class Node:
    def __init__(self, attribute):
        self.attribute = attribute
        self.children = []
        self.answer = ''

In [68]:
def build_tree(data, features):
    last_col = [row[-1] for row in data]
    if len(set(last_col)) == 1:
        node = Node('')
        node.answer = last_col[0]
        return node
    n = len(data[0]) - 1
    gains = [0]*n
    for col in range(n):
        gains[col] = compute_gain(data, col)
    split = gains.index(max(gains))
    node = Node(features[split])
    fea = features[:split] + features[split+1:]
    attr, dic = subtables(data, split, delete=True)
    for x in range(len(attr)):
        child = build_tree(dic[attr[x]], fea)
        node.children.append((attr[x], child))

    return node

In [69]:
def compute_gain(data, col):
    att_values, dic = subtables(data, col, delete=False)
    total_entropy = entropy([row[-1] for row in data])
    for x in range(len(att_values)):
        ratio = len(dic[att_values[x]]) / (len(data) * 1.0)
        entro = entropy([row[-1] for row in dic[att_values[x]]])
        total_entropy -= ratio*entro
    return total_entropy

In [74]:
def subtables(data, col, delete):
    dic = {}
    coldata = [row[col] for row in data]
    attr = list(set(coldata))
    counts = [0]*len(attr)
    r = len(data)
    c = len(data)
    for x in range(len(attr)):
        for y in range(r):
            if data[y][col] == attr[x]:
                counts[x] += 1
    for x in range(len(attr)):
        dic[attr[x]] = [[0 for i in range(c)] for j in range(counts[x])]
        pos = 0
        for y in range(r):
            if data[y][col] == attr[x]:
                if delete:
                    del data[y][col]
                dic[attr[x]][pos] = data[y]
                pos += 1
    return attr, dic

In [75]:
def entropy(S):
    attr = list(set(S))
    if len(attr) == 1:
        return 0
    counts = [0,0]
    for i in range(2):
        counts[i] = sum([1 for x in S if attr[i] == x]) / (len(S) * 1.0)
    sums = 0
    for cnt in counts:
        sums += -1 * cnt * math.log(cnt, 2)
    return sums

In [76]:
def print_tree(node, level):
    if node.answer != '':
        print(" "*level, node.answer)
        return
    print(" "*level, node.attribute)
    for value, n in node.children:
        print(" "*(level+1), value)
        print_tree(n, level+2)
    

In [82]:
dataset, features = load_csv('new_titanic.csv')
# print(dataset[:10], features)
node = build_tree(dataset[:20], features)
print('The decision tree is')
print_tree(node, 0)

The decision tree is
 sex
  Male
   pclass
    Third
     No
    Second
     Yes
    First
     No
  Female
   pclass
    Third
     pca
      Pcayes
       Yes
      Pcano
       age
        Child
         No
        Adult
         ssa
          Ssano
           Yes
          Ssayes
           No
    Second
     Yes
    First
     Yes


In [8]:
from random import randrange
from csv import reader
import math

# Load a CSV file
def load_csv(filename):
    file = open(filename, "r")
    lines = reader(file)
    dataset = list(lines)
    return dataset

# Split a dataset into a train and test set
def train_test_split(dataset, split=0.80):
    train = list()
    train_size = split * len(dataset)
    dataset_copy = list(dataset)
    while len(train) < train_size:
        index = randrange(len(dataset_copy))
        train.append(dataset_copy.pop(index))
    return train, dataset_copy
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a single split
def evaluate_algorithm_single(dataset, algorithm, *args):
    train_set,test_set=train_test_split(dataset,0.80)
    scores = list()
    predicted = algorithm(train_set, test_set, *args)
    actual = [row[-1] for row in test_set]
    accuracy = accuracy_metric(actual, predicted)
    scores.append(accuracy)
    return scores
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores
 
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right
 
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

# Calculate the Entropy for a split dataset
def entropy(groups, classes,b_score):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    ent = 0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            if p > 0 :
                score=(p*math.log(p,2))
        # weight the group score by its relative size i.e Entrpy gain
        ent-=(score*(size/n_instances))
    return ent

# Select the best split point for a dataset
def get_split(dataset,split_parameter):
    if split_parameter=='entropy':# this is invoked for parameter entropy
        class_values = list(set(row[-1] for row in dataset))
        b_index, b_value, b_score, b_groups = 999, 999, 1, None
        for index in range(len(dataset[0])-1):
            for row in dataset:
                groups = test_split(index, row[index], dataset)
                ent = entropy(groups, class_values,b_score)
                if ent < b_score:
                    b_index, b_value, b_score, b_groups = index, row[index], ent, groups
        return {'index':b_index, 'value':b_value, 'groups':b_groups}
    elif split_parameter=='gini':# this is invoked for parameter gini
        class_values = list(set(row[-1] for row in dataset))
        b_index, b_value, b_score, b_groups = 99999, 99999, 1, None
        for index in range(len(dataset[0])-1):
            for row in dataset:
                groups = test_split(index, row[index], dataset)
                gini = gini_index(groups, class_values)
                if gini < b_score:
                    b_index, b_value, b_score, b_groups = index, row[index], gini, groups
        return {'index':b_index, 'value':b_value, 'groups':b_groups}


def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left,split_parameter)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right,split_parameter)
        split(node['right'], max_depth, min_size, depth+1)


def build_tree(train, max_depth, min_size,split_parameter):
    root = get_split(train,split_parameter)
    split(root, max_depth, min_size, 1)
    return root

# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[ATTRIBUTE[%s] = %.50s]' % ((depth*'\t', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']


def decision_tree(train, test, max_depth, min_size,split_parameter):
    tree = build_tree(train, max_depth, min_size,split_parameter)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

# Datasets used tic_tac_toe,iris
# load and prepare data
filename = 'new_titanic.csv'#Provide the CSV filename on which you want to test
dataset = load_csv(filename)
# Tree model creation on training set
n_folds = 5
max_depth = 3
min_size = 1
split_parameter='entropy'  # 'entrpy'/'gini'
train_set,test_set=train_test_split(dataset,0.80)
tree= build_tree(train_set, max_depth, min_size,split_parameter)
print('Dictionary Representation of tree on training set')
print('  ')
print(tree)
print('  ')
print('Attributes ')
print(dataset[0])
print('Textual Representation of JSON tree')
print_tree(tree)
scores_1 = evaluate_algorithm_single(dataset,decision_tree,max_depth,min_size,split_parameter)
print('  ')
print('Implementing Single Split')
print('Scores: %s' % scores_1)
print('Accuracy: %.3f%%' % (sum(scores_1)/float(len(scores_1))))
#Calculating scores for k cross validation by setting n_folds value
print('  ')
print('Implementing k-cross validation')
scores_2 = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size,split_parameter)
print('Scores: %s' % scores_2)
print('Mean Accuracy: %.3f%%' % (sum(scores_2)/float(len(scores_2))))

#actual = [row[-1] for row in test_set]
#print(actual)
#predicted = decision_tree(train_set,test_set,max_depth, min_size)
#print(predicted)


Dictionary Representation of tree on training set
  
{'index': 1, 'value': 'Male', 'left': {'index': 0, 'value': 'Third', 'left': {'index': 5, 'value': 'Moderate', 'left': 'Yes', 'right': 'Yes'}, 'right': {'index': 5, 'value': 'Moderate', 'left': 'Yes', 'right': 'No'}}, 'right': {'index': 0, 'value': 'Second', 'left': {'index': 2, 'value': 'Senior_citizen', 'left': 'No', 'right': 'No'}, 'right': {'index': 4, 'value': 'Pcayes', 'left': 'No', 'right': 'No'}}}
  
Attributes 
['pclass', 'sex', 'age', 'ssa', 'pca', 'fare', 'survived']
Textual Representation of JSON tree
[ATTRIBUTE[2] = Male]
	[ATTRIBUTE[1] = Third]
		[ATTRIBUTE[6] = Moderate]
   [Yes]
   [Yes]
		[ATTRIBUTE[6] = Moderate]
   [Yes]
   [No]
	[ATTRIBUTE[1] = Second]
		[ATTRIBUTE[3] = Senior_citizen]
   [No]
   [No]
		[ATTRIBUTE[5] = Pcayes]
   [No]
   [No]
  
Implementing Single Split
Scores: [82.48587570621469]
Accuracy: 82.486%
  
Implementing k-cross validation
Scores: [79.09604519774011, 80.22598870056498, 80.7909604519774,

In [9]:
# def pos_and_neg(data):
#     positive,negative = 0,0
#     for d in data.iloc[:]['survived']:
#         if d == 'Yes':
#             positive += 1
#         else:
#             negative += 1
#     return positive, negative


# def cal_entropy(data):
#     positive, negative = pos_and_neg(data)
#     t1 = positive/(positive + negative)
#     t2 = negative/(positive + negative)
#     entropy = -(t1)*(math.log2(t1)) - (t2)*(math.log2(t2))
#     return entropy

# # cal_entropy(processed_data)


# columns = list(processed_data.columns)
# def func(data):
#     for col in columns:
#         for d in data.iloc[:][col]:
            
#             pass
# # func(processed_data)

In [17]:
from collections import Counter
df = pd.read_csv('processed_titanic.csv')
df.drop(['fare'],axis=1,inplace=True)
print(df)

def entropy(probs):  
    import math
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

def entropy_of_list(a_list):
    cnt = Counter(x for x in a_list)
    num_instances = len(a_list)*1.0
    probs = [x / num_instances for x in cnt.values()]
    return entropy(probs)

total_entropy = entropy_of_list(df['survived'])

print("\n Total Entropy of titanic Data Set:",total_entropy)

     pclass     sex    age     ssa     pca survived
0     third    male  adult  ssayes   pcano       no
1     first  female  adult  ssayes   pcano      yes
2     third  female  adult   ssano   pcano      yes
3     first  female  adult  ssayes   pcano      yes
4     third    male  adult   ssano   pcano       no
..      ...     ...    ...     ...     ...      ...
882  second    male  adult   ssano   pcano       no
883   first  female  adult   ssano   pcano      yes
884   third  female  child  ssayes  pcayes       no
885   first    male  adult   ssano   pcano      yes
886   third    male  adult   ssano   pcano       no

[887 rows x 6 columns]

 Total Entropy of titanic Data Set: 0.9618806789594468


In [18]:
def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    df_split = df.groupby(split_attribute_name)
    nobs = len(df.index) * 1.0
    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]
    df_agg_ent.columns = ['Entropy', 'po']
    
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['po'] )
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy


In [4]:

def id3(df, target_attribute_name, attribute_names, default_class=None):
    cnt = Counter(x for x in df[target_attribute_name])# class of YES /NO
    if len(cnt) == 1:
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class 
    else:
        default_class = max(cnt.keys()) 
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] #
        index_of_max = gainz.index(max(gainz))
        best_attr = attribute_names[index_of_max]
        tree = {best_attr:{}} # Iniiate the tree with best attribute as a node 
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]

        
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,target_attribute_name,remaining_attribute_names,default_class)
            tree[best_attr][attr_val] = subtree
        return tree

In [19]:
attribute_names = list(df.columns)
print("List of Attributes:", attribute_names) 
attribute_names.remove('survived') #Remove the class attribute 
print("Predicting Attributes:", attribute_names)

List of Attributes: ['pclass', 'sex', 'age', 'ssa', 'pca', 'survived']
Predicting Attributes: ['pclass', 'sex', 'age', 'ssa', 'pca']


In [20]:
from pprint import pprint
tree = id3(df[:50],'survived',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n")
pprint(tree)
attribute = next(iter(tree))
print("Best Attribute :\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())



The Resultant Decision Tree is :

{'sex': {'female': {'pclass': {'first': 'yes',
                               'second': {'age': {'adult': {'ssa': {'ssano': 'yes',
                                                                    'ssayes': 'no'}},
                                                  'child': 'yes'}},
                               'third': {'age': {'adult': {'ssa': {'ssano': 'yes',
                                                                   'ssayes': {'pca': {'pcano': 'no',
                                                                                      'pcayes': 'yes'}}}},
                                                 'child': {'ssa': {'ssano': {'pca': {'pcano': 'yes',
                                                                                     'pcayes': 'no'}},
                                                                   'ssayes': {'pca': {'pcano': 'yes',
                                                                                  

In [21]:
def classify(instance, tree, default=None): 
    attribute = next(iter(tree))
    if instance[attribute] in tree[attribute].keys(): # Value of the attributs in  set of Tree keys  
        result = tree[attribute][instance[attribute]]
        if isinstance(result, dict):
            return classify(instance, result)
        else:
            return result # this is a label
    else:
        return default

In [22]:
df['predicted'] = df.apply(classify, axis=1, args=(tree,'No') )

print('Accuracy is: ' + str(sum(df['survived']==df['predicted']) / (1.0*len(df.index))))

Accuracy is: 0.6335963923337091
